# Physical Units

Braincore includes a system for physical units. The base units are defined by their standard SI unit names:
`amp`/`ampere`, `kilogram`/`kilogramme`, `second`, `metre`/`meter`, `kilogram`, `mole`/`mol`, `kelvin`, and `candela`. In addition to these base units, braincore defines a set of derived units: `coulomb`, `farad`, `gram`/`gramme`, `hertz`, `joule`, `liter`/
`litre`, `molar`, `pascal`, `ohm`,  `siemens`, `volt`, `watt`,
together with prefixed versions (e.g. `msiemens = 0.001*siemens`) using the
prefixes `p, n, u, m, k, M, G, T` (two exceptions to this rule: `kilogram`
is not defined with any additional prefixes, and `metre` and `meter` are
additionaly defined with the "centi" prefix, i.e. `cmetre`/`cmeter`).
For convenience, a couple of additional useful standard abbreviations such as
`cm` (instead of `cmetre`/`cmeter`), `nS` (instead of `nsiemens`),
`ms` (instead of `msecond`), `Hz` (instead of `hertz`), `mM`
(instead of `mmolar`) are included. To avoid clashes with common variable
names, no one-letter abbreviations are provided (e.g. you can use `mV` or
`nS`, but *not* `V` or `S`).

## Importing units
Braincore generates standard names for units, combining the unit name (e.g. “siemens”) with a prefixes (e.g. “m”), and also generates squared and cubed versions by appending a number. For example, the units “msiemens”, “siemens2”, “usiemens3” are all predefined. You can import these units from the package `briancore.units` – accordingly, an `from braincore.units import *` will result in everything being imported.

We recommend importing only the units you need, to have a cleaner namespace. For example, `import braincore.units as U` and then using `bu.msiemens` instead of `msiemens`.

In [1]:
import brainunit as bu

## Using units
You can generate a physical quantity by multiplying a scalar or ndarray with its physical unit:

In [2]:
tau = 20 * bu.ms
tau

In [3]:
rates = [10, 20, 30] * bu.Hz
rates

In [4]:
rates = [[10, 20, 30], [20, 30, 40]] * bu.Hz
rates

Braincore will check the consistency of operations on units and raise an error for dimensionality mismatches:

In [5]:
try:
    tau += 1  # ms? second?
except Exception as e:
    print(e)

In [6]:
try:
    3 * bu.kgram + 3 * bu.amp
except Exception as e:
    print(e)

## Basics
Numpy functions have been overwritten to correctly work with units.

The important attributes of a `Quantity` object are:
- `value`: the numerical value of the quantity
- `unit`: the unit of the quantity
- `ndim`: the number of dimensions of quantity's value
- `shape`: the shape of the quantity's value
- `size`: the size of the quantity's value
- `dtype`: the dtype of the quantity's value

### An example

In [7]:
rates

In [8]:
rates.value

In [9]:
rates.unit

In [10]:
rates.ndim, rates.shape, rates.size, rates.dtype

### Quantity Creation
Creating a Quantity object can be accomplished in several ways, categorized based on the type of input used. Here, we present the methods grouped by their input types and characteristics for better clarity.

In [11]:
import jax.numpy as jnp
import brainstate as bst
bst.environ.set(precision=64) # we recommend using 64-bit precision for better numerical stability


#### Scalar and Array Multiplication
- Multiplying a Scalar with a Unit


In [12]:
5 * bu.ms

- Multiplying a Jax nunmpy value type with a Unit:

In [13]:
jnp.float64(5) * bu.ms

- Multiplying a Jax numpy array with a Unit:

In [14]:
jnp.array([1, 2, 3]) * bu.ms

- Multiplying a List with a Unit:

In [15]:
[1, 2, 3] * bu.ms

#### Direct Quantity Creation


- Creating a Quantity Directly with a Value

In [16]:
bu.Quantity(5)

- Creating a Quantity Directly with a Value and Unit

In [18]:
bu.Quantity(5, unit=bu.ms)

- Creating a Quantity with a Jax numpy Array of Values and a Unit

In [19]:
bu.Quantity(jnp.array([1, 2, 3]), unit=bu.ms)

- Creating a Quantity with a List of Values and a Unit

In [20]:
bu.Quantity([1, 2, 3], unit=bu.ms)

- Creating a Quantity with a List of Quantities

In [21]:
bu.Quantity([500 * bu.ms, 1 * bu.second])

- Using the with_units Method

In [22]:
bu.Quantity.with_units(jnp.array([0.5, 1]), second=1)

#### Unitless Quantity
Quantities can be unitless, which means they have no units. If there is no unit provided, the quantity is assumed to be unitless. The following are examples of creating unitless quantities:

In [23]:
bu.Quantity([1, 2, 3])

In [24]:
bu.Quantity(jnp.array([1, 2, 3]))

In [25]:
bu.Quantity([])

#### Illegal Quantity Creation
The following are examples of illegal quantity creation:

In [26]:
try:
    bu.Quantity([500 * bu.ms, 1])
except Exception as e:
    print(e)

In [27]:
try:
    bu.Quantity(["some", "nonsense"])
except Exception as e:
    print(e)

In [28]:
try:
    bu.Quantity([500 * bu.ms, 1 * bu.volt])
except Exception as e:
    print(e)

### Basic Operations
Like Numpy and Jax numpy, arithmetic operators on arrays apply elementwise. A new array is created and filled with the result.

In [29]:
a = [20, 30, 40, 50] * bu.mV
b = jnp.arange(4) * bu.mV
b

In [30]:
c = a - b
c

In [31]:
b = b ** 2
b

In [32]:
a < 35 * bu.mV

The product operator * operates elementwise in Quantity and the matrix product can be performed using the @ operator or the dot function or method:

In [33]:
A = jnp.array([[1, 2], [3, 4]]) * bu.mV
B = jnp.array([[2, 3], [4, 5]]) * bu.mV

A * B

In [34]:
A @ B

In [35]:
A.dot(B)

When operating with arrays of different types, the type of the resulting array corresponds to the more general or precise one

In [36]:
a = jnp.ones(3, dtype=jnp.int32) * bu.mV
b = jnp.linspace(0, jnp.pi, 3) * bu.mV

b.dtype.name

In [37]:
c = a + b
c.dtype.name

Many unary operations, such as computing the sum of all the elements in the array, are implemented as methods of the Quantity class.

In [38]:
a = bst.random.random((2, 3)) * bu.mV
a

In [39]:
a.sum()

In [40]:
a.min()

In [41]:
a.max()

By default, these operations apply to the array as though it were a list of numbers, regardless of its shape. However, by specifying the axis parameter you can apply an operation along the specified axis of a Quantity:

In [42]:
b = jnp.arange(12).reshape(3, 4) * bu.mV
b

In [43]:
b.sum(axis=0) # sum of each column

In [44]:
b.min(axis=1) # min of each row

In [45]:
b.cumsum(axis=1) # cumulative sum along each row

### Universal Functions
Braincore provides familiar mathematical functions such as sin, cos, and exp. In braincore, these are called “universal functions” (ufunc). Within braincore, these functions operate elementwise on a Quantity, producing a Quantity as output.
Attention: For some ufuncs, they only work with unitless quantities. If you want to know the category of a ufunc, you can check the `braincore.math.ufuncs.py`.

In [46]:
B = bu.Quantity(jnp.arange(3))
B

In [47]:
bu.math.exp(B)

In [48]:
bu.math.sqrt(B)

In [49]:
C = bu.Quantity(jnp.arange(3))
bu.math.add(B, C)

### Indexing, Slicing, and Iterating
One-dimensional Quantity can be indexed, sliced and iterated over, much like lists and other Python sequences.

In [50]:
a = jnp.arange(10) ** 3 * bu.mV
a

In [51]:
a[2]

In [52]:
a[2:5]

Only same dimension Quantity can be set to a slice of a Quantity.

In [53]:
# equivalent to a[0:6:2] = 1000;
# from start to position 6, exclusive, set every 2nd element to 1000
a[:6:2] = 1000 * bu.mV
a

In [54]:
a[::-1] # reversed a

In [55]:
for i in a:
    print(i**(1 / 3.))

Multidimensional Quantity can have one index per axis. These indices are given in a tuple separated by commas:

In [56]:
def f(x, y):
    return 10 * x + y
b = jnp.fromfunction(f, (5, 4), dtype=jnp.int32) * bu.mV
b

In [57]:
b[2, 3]

In [58]:
b[0:5, 1]  # each row in the second column of b

In [59]:
b[:, 1]  # equivalent to the previous example

In [60]:
b[1:3, :]  # each column in the second and third row of b

When fewer indices are provided than the number of axes, the missing indices are considered complete slices:

In [61]:
b[-1]

The expression within brackets in b[i] is treated as an i followed by as many instances of : as needed to represent the remaining axes. NumPy also allows you to write this using dots as b[i, ...].

The dots (...) represent as many colons as needed to produce a complete indexing tuple. For example, if x is a Quantity with 5 axes, then
- x[1, 2, ...] is equivalent to x[1, 2, :, :, :],
- x[..., 3] to x[:, :, :, :, 3] and
- x[4, ..., 5, :] to x[4, :, :, 5, :].

In [62]:
c = jnp.array([[[0, 1, 2], [10, 12, 13]], [[100, 101, 102], [110, 112, 113]]]) * bu.mV # a 3D array (two stacked 2D arrays)
c.shape

In [63]:
c[1, ...] # same as c[1, :, :] or c[1]

In [64]:
c[..., 2] # same as c[:, :, 2]

Iterating over multidimensional Quantity is done with respect to the first axis:

In [65]:
for row in b:
    print(row)

## Shape Manipulation
### Changing the shape of a Quantity
A Quantity has a shape given by the number of elements along each axis:

In [66]:
a = jnp.floor(10 * bst.random.random((3, 4))) * bu.mV
a

In [67]:
a.shape

The shape of a Quantity can be changed with various commands. Note that the following three commands all return a modified array, but do not change the original array:

In [68]:
a.ravel()  # returns the array, flattened

In [69]:
a.reshape(6, 2) # returns the array with a modified shape

In [70]:
a.T  # returns the array, transposed

In [71]:
a.T.shape

In [72]:
a.shape

The order of the elements in the Quantity resulting from ravel is normally “C-style”, that is, the rightmost index “changes the fastest”, so the element after a[0, 0] is a[0, 1]. If the Quantity is reshaped to some other shape, again the Quantity is treated as “C-style”. NumPy normally creates arrays stored in this order, so ravel will usually not need to copy its argument, but if the Quantity was made by taking slices of another Quantity or created with unusual options, it may need to be copied. The functions ravel and reshape can also be instructed, using an optional argument, to use FORTRAN-style arrays, in which the leftmost index changes the fastest.

The reshape function only returns its argument with a modified shape, due to Jax's immutability. The resize method is not available in braincore.

If a dimension is given as -1 in a reshaping operation, the other dimensions are automatically calculated:

In [73]:
a.reshape(3, -1)

### Stacking together different arrays
Several arrays can be stacked together along different axes:

In [74]:
a = jnp.floor(10 * bst.random.random((2, 2))) * bu.mV
a

In [75]:
b = jnp.floor(10 * bst.random.random((2, 2))) * bu.mV
b

In [76]:
bu.math.vstack((a, b))

In [77]:
bu.math.hstack((a, b))

The function column_stack stacks 1D Quantities as columns into a 2D Quantities. It is equivalent to hstack only for 2D Quantities:

In [78]:
bu.math.column_stack((a, b)) # with 2D arrays

In [79]:
a = jnp.array([4., 2.]) * bu.mV
b = jnp.array([2., 8.]) * bu.mV
bu.math.column_stack((a, b)) # returns a 2D array

In [80]:
bu.math.hstack((a, b)) # the result is different

In [81]:
a[:, jnp.newaxis] # view `a` as a 2D column vector

In [82]:
bu.math.column_stack((a[:, jnp.newaxis], b[:, jnp.newaxis]))

In [83]:
bu.math.hstack((a[:, jnp.newaxis], b[:, jnp.newaxis])) # the result is the same

### Splitting one Quantity into several smaller ones
Using hsplit, you can split an array along its horizontal axis, either by specifying the number of equally shaped Quantities to return, or by specifying the columns after which the division should occur:

In [84]:
a = jnp.floor(10 * bst.random.random((2, 12))) * bu.mV
a

In [85]:
bu.math.hsplit(a, 3) # Split a into 3

In [86]:
bu.math.hsplit(a, (3, 4)) # Split `a` after the third and the fourth column

vsplit splits along the vertical axis, and array_split allows one to specify along which axis to split.

## Advanced Guide

### Displaying Quantity
Braincore provide `in_unit` method to display the Quantity in a specific unit. The following are examples of displaying Quantity in a specific unit:

In [87]:
q = 3. * bu.volt
q

In [88]:
bu.in_unit(q, bu.mvolt)

### Unary Operations
The unary operations positive (+) ,negative (-), absolute (abs), and inversion (~) are supported:

In [89]:
q = 5. * bu.mV
q

In [90]:
q = -q
q

In [91]:
q = +q
q

In [92]:
q = abs(q)
q

In [93]:
q1 = bu.Quantity(0b101)
q2 = bu.Quantity(-0b110)
q1, q2

In [94]:
~q1, ~q2

### Comparison
The comparison operators (<, <=, ==, !=, >, >=) are supported:

In [95]:
q1 = jnp.arange(10, 20, 2) * bu.mV
q2 = jnp.arange(8, 27, 4) * bu.mV
q1, q2

In [96]:
q1 == q2, q1 != q2

In [97]:
q1 < q2, q1 <= q2

In [98]:
q1 > q2, q1 >= q2

### Binary Operations
The binary operations add (+), subtract (-), multiply (*), divide (/), floor divide (//), remainder (%), divmod (divmod), power (**), matmul (@), shift (<<, >>), round(round) are supported:

In [99]:
q1 = jnp.array([1, 2, 3]) * bu.mV
q2 = jnp.array([2, 3, 4]) * bu.mV
q1, q2

In [100]:
q1 + q2, q1 - q2

In [101]:
q1 * q2

In [102]:
q1 / q2, q1 // q2, q1 % q2

In [103]:
divmod(q1, q2)

In [104]:
q1 ** 2

In [105]:
q1 @ q2

In [106]:
q1 = bu.Quantity(jnp.arange(5, dtype=jnp.int32), unit=bu.mV.unit)
q1 << 2, q1 >> 2

In [107]:
q1 = 80.23456 * bu.mV
q2 = 100.000056 * bu.mV
q3 = -100.000056 * bu.mV
print("round(80.23456, 2) : ", q1.round(5))
print("round(100.000056, 3) : ", q2.round(6))
print("round(-100.000056, 3) : ", q3.round(6))

### Shape Manipulation
The shape of an array can be changed with various commands. Note that the following three commands all return a modified array, but do not change the original array:

In [108]:
q = [[1, 2], [3, 4]] * bu.mV
q

In [109]:
q.flatten()

In [110]:
q.swapaxes(0, 1)

In [111]:
q.take(jnp.array([0, 2]))

In [112]:
q.transpose()

In [113]:
q.tile(2)

In [114]:
q.unsqueeze(0)

In [115]:
q.expand_dims(0)

In [116]:
expand_as_shape = (1, 2, 2)
q.expand_as(jnp.zeros(expand_as_shape).shape)

In [117]:
q_put = [[1, 2], [3, 4]] * bu.mV
q_put.put([[1, 0], [0, 1]], [10, 30] * bu.mV)
q_put

In [118]:
q_squeeze = [[1, 2], [3, 4]] * bu.mV
q_squeeze.squeeze()

In [119]:
q_spilt = [[1, 2], [3, 4]] * bu.mV
q_spilt.split(2)

### Numpy Methods
All methods that make sense on quantities should work, i.e. they check for the correct units of their arguments and return quantities with units were appropriate.

These methods defined at `braincore.math`, so you can use them by importing `import braincore.math as bm` and then using `bm.method_name`.
#### Functions that remove unit
- all
- any
- nonzero
- argmax
- argmin
- argsort
- ones_like
- zeros_like

#### Functions that keep unit
- round
- std
- sum
- trace
- cumsum
- diagonal
- max
- mean
- min
- ptp
- ravel
- absolute
- rint
- negative
- positive
- conj
- conjugate
- floor
- ceil
- trunc

#### Functions that change unit
- var
- multiply
- divide
- true_divide
- floor_divide
- dot
- matmul
- sqrt
- square
- reciprocal

#### Functions that need to match unit
- add
- subtract
- maximum
- minimum
- remainder
- mod
- fmod

#### Functions that only work with unitless quantities
- sin
- sinh
- arcsinh
- cos
- cosh
- arccos
- arccosh
- tan
- tanh
- arctan
- arctanh
- log
- log10
- exp
- expm1
- log1p

#### Functions that compare quantities
- less
- less_equal
- greater
- greater_equal
- equal
- not_equal

#### Functions that work on all quantities and return boolean arrays(Logical operations)
- logical_and
- logical_or
- logical_xor
- logical_not
- isreal
- iscomplex
- isfinite
- isinf
- isnan